In [89]:
import pickle
import pandas as pd

In [107]:
lr = pickle.load(open("../lr_model.pkl", 'rb'))

In [108]:
test = pd.read_pickle("../data/hurricane_michael.pkl")

In [109]:
test.head()

,date,favorites,geo,hashtags,id,mentions,permalink,retweets,text,to,username
0,2018-10-12 23:59:57+00:00,4,,#HurricaneMichael #blessed #UnitedWeStand #tallahasseeproud,1050898900582838272,@COTNews,https://twitter.com/joeearenas/status/1050898900582838272,0,"@COTNews has been working 24/7 to restore services in Tallahassee. Thank you, to all the voulenteers and people of this city for everything. You are our heros! #HurricaneMichael #blessed #UnitedWeStand #tallahasseeproud",None,joeearenas
1,2018-10-12 23:59:53+00:00,1,,,1050898882526371842,,https://twitter.com/LakesideBexley/status/1050898882526371842,0,"In the wake of Hurricane Michael, we understand the importance of knowing where to find aid. For storm recovery resources, please visit the following link: https://feaweb.org/hurricane-michael-resources-and-relief …https://feaweb.org/hurricane-michael-resources-and-relief …",None,LakesideBexley
2,2018-10-12 23:59:52+00:00,0,,#HurricaneMichael #Florida,1050898882077442048,,https://twitter.com/PRAISETRIUNEGOD/status/1050898882077442048,0,"Maybe 17 "" #HurricaneMichael Updates: Body Found on #Florida Beach with Death Toll Already at 16 ""https://weather.com/amp/storms/hurricane/news/2018-10-12-hurricane-michael-impacts-death-toll-southeast-mid-atlantic.html …",None,PRAISETRIUNEGOD
3,2018-10-12 23:59:49+00:00,0,,,1050898865988222976,,https://twitter.com/aShartee/status/1050898865988222976,0,In other news praying for those affected by hurricane Michael,None,aShartee
4,2018-10-12 23:59:43+00:00,0,,#HurricaneMichael #ExcessiveForce,1050898841879236608,,https://twitter.com/MindOfMo/status/1050898841879236608,0,"iSpy 2 or 3 who'd be hard-pressed to RUN in an emergency. I mention it cuz: 1. POTUS is not honoring #HurricaneMichael, WTF. 2. Suspect overweight/unfit contributes to #ExcessiveForce, including/especially fatally shooting FAST. 3. LEO mostly giving Heir carte blanche (like MBS).https://twitter.com/CBSNews/status/1050891575067377664 …",CBSNews,MindOfMo


In [110]:
tweets = test[['text','permalink']]

In [94]:
tweets = tweets.rename(columns={"text":"tweet_text"})

In [95]:
#make tweets lowercase
lower_tweets = []
for tweet in tweets.tweet_text:
    lower_tweets.append(tweet.lower())

lower_tweets = pd.DataFrame(lower_tweets)

lower_tweets.rename(columns={0:"tweet"}, inplace=True)

tweets.tweet_text = lower_tweets.tweet

#truncate elongations (NO CODE FOR NOW)
#>>> import re
#>>> re.sub(r'(.)\1+', r'\1\1', "haaaaapppppyyy")  #https://stackoverflow.com/questions/10072744/remove-repeating-characters-from-words
#spell out numbers (NO CODE FOR NOW)

#replace all usernames with 'USERID'
tweets.tweet_text = tweets.tweet_text.replace(r'@\S+', 'USERID', regex=True)

#remove 'USERID'
userid_tweets = []
for tweet in tweets.tweet_text:
    userid_tweets.append(tweet.replace('USERID', ''))

userid_tweets = pd.DataFrame(userid_tweets)

userid_tweets.rename(columns={0:"tweet"}, inplace=True)
tweets.tweet_text = userid_tweets.tweet


#replace all URLs with HTTP
tweets.tweet_text = tweets.tweet_text.replace(r'http\S+', 'HTTP', regex=True).replace(r'www\S+', 'HTTP', regex=True)

#remove HTTP
http_tweets = []
for tweet in tweets.tweet_text:
    http_tweets.append(tweet.replace('HTTP', ''))

http_tweets = pd.DataFrame(http_tweets)

http_tweets.rename(columns={0:"tweet"}, inplace=True)
tweets.tweet_text = http_tweets.tweet

#remove "RT" string
tweets.tweet_text = tweets.tweet_text.map(lambda x: x.lstrip('rt'))

#remove remaining punctuation except for "#"
tweets.tweet_text = tweets.tweet_text.str.replace("[^a-zA-Z]", " ")

In [96]:
tweets = tweets.drop_duplicates(subset=["tweet_text"])

In [97]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer

In [98]:
tknzr = TweetTokenizer()
cvec = CountVectorizer(stop_words="english", tokenizer=tknzr.tokenize, max_features=14897)
test = cvec.fit_transform(tweets['tweet_text'])

In [99]:
predictions = lr.predict(test)

In [100]:
tweets["predictions"] = predictions

In [101]:
tweets.head()

,tweet_text,id,predictions
0,has been working to restore services in tallahassee thank you to all the voulenteers and people of this city for everything you are our heros hurricanemichael blessed unitedwestand tallahasseeproud,1050898900582838272,0
1,in the wake of hurricane michael we understand the importance of knowing where to find aid for storm recovery resources please visit the following link,1050898882526371842,1
2,maybe hurricanemichael updates body found on florida beach with death toll already at,1050898882077442048,0
3,in other news praying for those affected by hurricane michael,1050898865988222976,0
4,ispy or who d be hard pressed to run in an emergency i mention it cuz potus is not honoring hurricanemichael wtf suspect overweight unfit contributes to excessiveforce including especially fatally shooting fast leo mostly giving heir carte blanche like mbs,1050898841879236608,0


In [102]:
tweets.predictions.value_counts()

0    30246
1    11476
Name: predictions, dtype: int64

In [103]:
pd.set_option('display.max_colwidth', -1)

In [104]:
sample_tweets = tweets[tweets.predictions == 1][['tweet_text','permalink']][:10]

In [105]:
sample_tweets

,tweet_text,id
1,in the wake of hurricane michael we understand the importance of knowing where to find aid for storm recovery resources please visit the following link,1050898882526371842
7,hurricane michael leaves friends relatives searching for thousands of people miami herald,1050898807544864775
11,sorry folks we are located in tallahassee florida and were hit by hurricane michael pic twitter com izjgduxxu,1050898753190862850
13,short work trip to atl was fast and furious thankfully the hurricane avoided us hurricanemichael atl atlanta lightposts symmetry vsco vscocam rain trees,1050898746404470785
15,o all victims of hurricane michael alive or no more my hope and prayers will go out to all of you cling onto hope and don t give in,1050898709913964544
16,sorry folks we are located in tallahassee florida and were hit by hurricane michael,1050898645414014976
17,i liked a video hurricane michael causes the unthinkable,1050898645195853824
36,i liked a video a message from president trump on hurricane michael,1050898318782533632
40,widespread catastrophic damage every building at tyndall afb totaled by hurricane michael,1050898256639811585
42,devastated beyond words panama city the day after hurricane michael,1050898216131198976


In [88]:
sample_tweets.to_csv("../data/sample_tweets.csv")